In [1]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [2]:
import os
import json
import pandas as pd
import time
from pinecone import Pinecone as pcn
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAI
from IPython.display import Markdown

/home/acrisvall/recipes_rag/.venv/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### 01 Get API Keys

In [3]:
api_key = os.environ.get("PINECONE_API_KEY")
openai_api_key = os.getenv('OPENAI_API_KEY')

In [4]:
# configure client
pc = pcn(api_key=api_key)
index_name = 'recipes-index'
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1106}},
 'total_vector_count': 1106}

### 02 Set Embedding Model

In [5]:
embed_model = "text-embedding-3-small"
embed = OpenAIEmbeddings(model=embed_model)

In [18]:
text_field = "description"

vectorStore = PineconeVectorStore(
  index, embed, text_field
)

### 03 Vector by Similarity Search

In [115]:
query = "Pain au chocolat"
query_res = vectorStore.similarity_search(query, k=3)

# Imprimir el metadata estructurado
for i, doc in enumerate(query_res):
    print(f"Document {i + 1}:")
    for key, value in doc.metadata.items():
        print(f"  {key}: {value}")
    print()  # Para separar cada documento visualmente


Document 1:
  calories: 540
  carbs: 54g
  cook_time: 15 mins
  diet_type: Nut-free
  difficulty: Easy
  fat: 31g
  fibre: 4g
  id: 428
  ingredients: 420gplain flour, 8gsea salt, 60gcaster sugar, 220gunsalted butter(at least 82% fat works best), 20g softened, 20gfresh yeast, or 10g dried yeast, 1egg, beaten, 110mlwhole milk, 16dark chocolate batons
  instructions: Make the dough in astand mixer. Combine the flour, salt, sugar and 20g softened butter. Dissolve the yeast in around 80ml tepid water, around 38C, not too hot or it will kill the yeast. Add 20g of beaten egg and the milk to the stand mixer bowl. Pour in the yeast-water mixture and turn on to a low speed with a dough hook attachment for approximately 5 mins until the dough comes together to form a rough ball. Increase the speed to medium-high and knead for a further 6-10 mins until the dough is smooth. The exact time will depend on your stand mixer. Wrap the dough tightly and rest at room temperature for 10 mins. Unwrap and r

### 04 Retrieve relevant contexts

In [81]:
# query = "please a recipe that can be prepare in less than 20 min"
# query = "please a dessert and a salad recommendation"
query = "Pain au chocolat"
# create the query vector
query_vector = embed.embed_query(query)
# Retrieve relevant contexts from Pinecone
query_res = index.query(vector=query_vector, top_k=3, include_metadata=True)
print(query_res)
print(len(query_res["matches"]))

# Extract the matched metadata (recipes) from Pinecone results
contexts = [item['metadata'] for item in query_res['matches']]

{'matches': [{'id': '428',
              'metadata': {'calories': '540',
                           'carbs': '54g',
                           'cook_time': '15 mins',
                           'description': 'Try your hand at making these '
                                          'popular French breakfast pastries '
                                          "filled with chocolate. They're "
                                          'perfect served with a cup of '
                                          'coffee, or as a mid-morning treat',
                           'diet_type': 'Nut-free',
                           'difficulty': 'Easy',
                           'fat': '31g',
                           'fibre': '4g',
                           'id': '428',
                           'ingredients': '420gplain flour, 8gsea salt, '
                                          '60gcaster sugar, 220gunsalted '
                                          'butter(at least 82% fat works '
  

In [107]:
context_texts = [
    f"Title: {c.get('title', 'N/A')}\n"
    f"Description: {c.get('description', 'N/A')}\n"
    f"Preparation Time: {c.get('prep_time', 'N/A')}\n"
    f"Cooking Time: {c.get('cook_time', 'N/A')}\n"
    f"Difficulty: {c.get('difficulty', 'N/A')}\n"
    f"Serves: {c.get('serves', 'N/A')}\n"
    f"Diet Type: {c.get('diet_type', 'N/A')}\n"
    f"Nutrition Facts: {c.get('calories', 'N/A')} | {c.get('carbs', 'N/A')} kcal | fat {c.get('fat', 'N/A')} | protein {c.get('protein', 'N/A')} | fibre {c.get('fibre', 'N/A')}\n"
    f"Ingredients: {c.get('ingredients', 'N/A')}\n"
    f"Instructions: {c.get('instructions', 'N/A')}"
    for c in contexts
]
augmented_query = "\n\n---\n\n".join(context_texts) + "\n\n-----\n\n" + query
print(augmented_query)

Title: Pain au chocolat
Description: Try your hand at making these popular French breakfast pastries filled with chocolate. They're perfect served with a cup of coffee, or as a mid-morning treat
Preparation Time: 45 mins
Cooking Time: 15 mins
Difficulty: Easy
Serves: Serves 8
Diet Type: Nut-free
Nutrition Facts: 540 | 54g kcal | fat 31g | protein 8g | fibre 4g
Ingredients: 420gplain flour, 8gsea salt, 60gcaster sugar, 220gunsalted butter(at least 82% fat works best), 20g softened, 20gfresh yeast, or 10g dried yeast, 1egg, beaten, 110mlwhole milk, 16dark chocolate batons
Instructions: Make the dough in astand mixer. Combine the flour, salt, sugar and 20g softened butter. Dissolve the yeast in around 80ml tepid water, around 38C, not too hot or it will kill the yeast. Add 20g of beaten egg and the milk to the stand mixer bowl. Pour in the yeast-water mixture and turn on to a low speed with a dough hook attachment for approximately 5 mins until the dough comes together to form a rough bal

In [156]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_openai import ChatOpenAI 
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

# completion llm  
llm = ChatOpenAI(  
    model_name='gpt-4o-mini',  
    temperature=0.0  
) 

retriever = vectorStore.as_retriever(search_type="similarity", k=3)
# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
  """
    Always add a kind and friendly message according to the context at the beginning of the response.
    Take the information from context[0]["metadata"]
    Then follow this format
    Title: [Title of the Recipe]
    Description: [Short Description of the Recipe]
    Preparation Time: [Preparation Time]
    Cooking Time: [Cooking Time]
    Difficulty: [Difficulty Level]
    Serves: [Number of Servings]
    Diet Type: [Diet Type]
    Nutrition: [List of Nutrition Facts]
    Ingredients: [List of Ingredients]
    Instructions: [Step-by-step Cooking Instructions]

    If you don't have any general information, just respond with "I don't know!"

    {context}
    {{metadata}}
    """
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "Give the 03 recipes with spaghetti"})
print(response)
response["answer"]
display(Markdown(response["answer"]))

{'input': 'Give the 03 recipes with spaghetti', 'context': [Document(id='43', metadata={'calories': '517', 'carbs': '65g', 'cook_time': '35 mins', 'diet_type': 'Dairy-free', 'difficulty': 'Easy', 'fat': '18g', 'fibre': '15g', 'id': '43', 'ingredients': '350gwholemeal spaghetti, 2 tbsprapeseed oil, 2aubergines(about 550g), each sliced into 4 lengthways, then cut into cubes, 4large garlic cloves,finely chopped, 3 tbsptomato purée, 1 tbspbalsamic vinegar, 250mlvegetable stock,made with 1 tbsp bouillon powder (vegan, if needed), 330gcherry tomatoes,halved, 30gpack of basil,chopped, 1 tbspcapers, 50gwalnut pieces,lightly toasted and chopped if large', 'instructions': 'Cook the spaghetti following pack instructions, then drain, reserving 150ml of the cooking water. Meanwhile, heat the oil in a large non-stick pan over a medium heat. Add the aubergine and garlic, stir well, then cover and cook for 8-10 mins, stirring a few times. Mix in the tomato purée and vinegar, and cook for 2-3 mins more

Hello! I'm happy to share some delicious spaghetti recipes with you. Here are three wonderful options:

---

**Title:** Spaghetti with Aubergine, Tomatoes, Capers, and Walnuts  
**Description:** A fibre-rich, healthy, and filling lunch that's part of our Healthy Diet Plan.  
**Preparation Time:** 10 minutes  
**Cooking Time:** 20 minutes  
**Difficulty:** Easy  
**Serves:** 4  
**Diet Type:** Vegetarian  
**Nutrition:** High in fiber, vitamins, and healthy fats  
**Ingredients:**  
- Spaghetti  
- Aubergine (eggplant)  
- Tomatoes  
- Capers  
- Walnuts  
- Olive oil  
- Garlic  
- Fresh parsley  
- Salt and pepper  

**Instructions:**  
1. Cook spaghetti according to package instructions.  
2. In a pan, heat olive oil and sauté garlic until fragrant.  
3. Add diced aubergine and cook until soft.  
4. Stir in chopped tomatoes, capers, and walnuts.  
5. Season with salt and pepper, and cook for another 5 minutes.  
6. Toss the cooked spaghetti with the sauce and garnish with fresh parsley.

---

**Title:** Super-Speedy Spaghetti with Garlic, Parsley, and Bacon  
**Description:** A super-simple and super-speedy spaghetti supper, perfect for days when your fridge is bare.  
**Preparation Time:** 5 minutes  
**Cooking Time:** 15 minutes  
**Difficulty:** Easy  
**Serves:** 2  
**Diet Type:** Non-Vegetarian  
**Nutrition:** High in protein and flavor  
**Ingredients:**  
- Spaghetti  
- Bacon  
- Garlic  
- Fresh parsley  
- Olive oil  
- Salt and pepper  

**Instructions:**  
1. Cook spaghetti according to package instructions.  
2. In a pan, cook bacon until crispy, then remove and set aside.  
3. In the same pan, sauté minced garlic in olive oil.  
4. Add the cooked spaghetti and toss to coat.  
5. Stir in chopped bacon and fresh parsley.  
6. Season with salt and pepper before serving.

---

**Title:** Spaghetti with Nduja, Spring Greens, and Breadcrumb Topping  
**Description:** A simple and speedy dish, ideal for a midweek meal.  
**Preparation Time:** 10 minutes  
**Cooking Time:** 15 minutes  
**Difficulty:** Easy  
**Serves:** 4  
**Diet Type:** Non-Vegetarian  
**Nutrition:** Rich in protein and vitamins  
**Ingredients:**  
- Spaghetti  
- Nduja (spicy spreadable sausage)  
- Spring greens (like spinach or kale)  
- Breadcrumbs  
- Olive oil  
- Parmesan cheese (optional)  
- Salt and pepper  

**Instructions:**  
1. Cook spaghetti according to package instructions.  
2. In a pan, heat nduja until it melts.  
3. Add spring greens and cook until wilted.  
4. Combine with the cooked spaghetti and mix well.  
5. In a separate pan, toast breadcrumbs in olive oil until golden.  
6. Serve the spaghetti topped with toasted breadcrumbs and grated Parmesan if desired.

---

I hope you enjoy trying out these recipes! Happy cooking!

## 05 Llm completion

In [136]:
from langchain_openai import ChatOpenAI  
from langchain.chains import RetrievalQA  


text = "please recommend 2 recipes"

query = f"""
Please respond the following query:{text}
In the respond follow this instructions:
  Don't Thank you for the additional context or mentioned the additional context. 
  Always add a Kind and friendly message according to the context at the beginning of the response like this examples:
  "I recommend", "Happy to help you", "I will help you", "sure! this is..."
  If any specific information is missing, omit that field.
  Then follow then this format for the recipe:
    Title: [Title of the Recipe]
    Description: [Short Description of the Recipe]
    Preparation Time: [Preparation Time]
    Cooking Time: [Cooking Time]
    Difficulty: [Difficulty Level]
    Serves: [Number of Servings]
    Diet Type: [Diet Type]
    Nutrition: [List of Nutrition Facts like calories, fibre, fat, protein]
    Ingredients: [List of Ingredients]
    Instructions: [Step-by-step Cooking Instructions]

  Make sure to omit any field where you don't have information.
  If you don't have any general information, just respond with "I don't know!"

"""

# completion llm  
llm = ChatOpenAI(  
    model_name='gpt-4o-mini',  
    temperature=0.0  
)  
qa = RetrievalQA.from_chain_type(  
    llm=llm,  
    chain_type="refine",  # use stuff to simple and direct output
    retriever=vectorStore.as_retriever(
      search_type="similarity",
      search_kwargs={"k": 2},
    )  
)  
res = qa.invoke(query)  
print(res)
display(Markdown(res["query"]))
display(Markdown(res["result"]))


{'query': '\nPlease respond the following query:please recommend 2 recipes\nIn the respond follow this instructions:\n  Don\'t Thank you for the additional context or mentioned the additional context. \n  Always add a Kind and friendly message according to the context at the beginning of the response like this examples:\n  "I recommend", "Happy to help you", "I will help you", "sure! this is..."\n  If any specific information is missing, omit that field.\n  Then follow then this format for the recipe:\n    Title: [Title of the Recipe]\n    Description: [Short Description of the Recipe]\n    Preparation Time: [Preparation Time]\n    Cooking Time: [Cooking Time]\n    Difficulty: [Difficulty Level]\n    Serves: [Number of Servings]\n    Diet Type: [Diet Type]\n    Nutrition: [List of Nutrition Facts like calories, fibre, fat, protein]\n    Ingredients: [List of Ingredients]\n    Instructions: [Step-by-step Cooking Instructions]\n\n  Make sure to omit any field where you don\'t have inform


Please respond the following query:please recommend 2 recipes
In the respond follow this instructions:
  Don't Thank you for the additional context or mentioned the additional context. 
  Always add a Kind and friendly message according to the context at the beginning of the response like this examples:
  "I recommend", "Happy to help you", "I will help you", "sure! this is..."
  If any specific information is missing, omit that field.
  Then follow then this format for the recipe:
    Title: [Title of the Recipe]
    Description: [Short Description of the Recipe]
    Preparation Time: [Preparation Time]
    Cooking Time: [Cooking Time]
    Difficulty: [Difficulty Level]
    Serves: [Number of Servings]
    Diet Type: [Diet Type]
    Nutrition: [List of Nutrition Facts like calories, fibre, fat, protein]
    Ingredients: [List of Ingredients]
    Instructions: [Step-by-step Cooking Instructions]

  Make sure to omit any field where you don't have information.
  If you don't have any general information, just respond with "I don't know!"



Happy to help you! Here are two recipes inspired by classic Indian dhal:

**Title:** Classic Indian Dhal  
**Description:** A flavorful and nutritious Indian dhal, known for its Ayurvedic properties, easy to prepare, and packed with spices.  
**Preparation Time:** 10 minutes  
**Cooking Time:** 30 minutes  
**Difficulty:** Easy  
**Serves:** 4  
**Diet Type:** Vegetarian  
**Nutrition:** Calories: 250, Fiber: 8g, Fat: 4g, Protein: 12g  
**Ingredients:**  
- 1 cup red lentils (masoor dal)  
- 4 cups water  
- 1 onion, chopped  
- 2 tomatoes, chopped  
- 2 cloves garlic, minced  
- 1-inch piece ginger, grated  
- 1 tsp turmeric powder  
- 1 tsp cumin seeds  
- 1 tsp coriander powder  
- 1 tsp garam masala  
- 2 tbsp oil or ghee  
- Salt to taste  
- Fresh cilantro for garnish  

**Instructions:**  
1. Rinse the red lentils under cold water until the water runs clear.  
2. In a pot, heat oil or ghee over medium heat. Add cumin seeds and let them sizzle.  
3. Add chopped onion, garlic, and ginger, sautéing until the onion is translucent.  
4. Stir in the chopped tomatoes, turmeric, coriander powder, and salt. Cook until the tomatoes soften.  
5. Add the rinsed lentils and water, bringing to a boil. Reduce heat and simmer for about 20 minutes, or until the lentils are soft.  
6. Stir in garam masala and adjust seasoning if needed.  
7. Garnish with fresh cilantro and serve warm with rice or naan.

---

**Title:** Spiced Vegetable Dhal  
**Description:** A hearty and nutritious dhal enriched with a variety of vegetables and spices, perfect for a wholesome meal.  
**Preparation Time:** 15 minutes  
**Cooking Time:** 30 minutes  
**Difficulty:** Easy  
**Serves:** 4  
**Diet Type:** Vegetarian  
**Nutrition:** Calories: 300, Fiber: 10g, Fat: 5g, Protein: 14g  
**Ingredients:**  
- 1 cup yellow split peas (toor dal)  
- 4 cups vegetable broth  
- 1 carrot, diced  
- 1 cup spinach, chopped  
- 1 onion, chopped  
- 2 cloves garlic, minced  
- 1-inch piece ginger, grated  
- 1 tsp turmeric powder  
- 1 tsp cumin seeds  
- 1 tsp mustard seeds  
- 1 tsp chili powder (optional)  
- 2 tbsp oil  
- Salt to taste  

**Instructions:**  
1. Rinse the yellow split peas under cold water until the water runs clear.  
2. In a pot, heat oil over medium heat. Add mustard seeds and cumin seeds, letting them pop.  
3. Add chopped onion, garlic, and ginger, sautéing until the onion is soft.  
4. Stir in the diced carrot, turmeric, and chili powder, cooking for a few minutes.  
5. Add the rinsed split peas and vegetable broth, bringing to a boil. Reduce heat and simmer for about 25 minutes, or until the peas are tender.  
6. Stir in the chopped spinach and season with salt. Cook for an additional 5 minutes.  
7. Serve warm with rice or bread.